<a href="https://colab.research.google.com/github/amirda9/ALLofMyDataGatherings/blob/master/image_detecion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Live Camera

In [ ]:
# import tensorflow_hub as hub
# import cv2
# import numpy
# import tensorflow as tf
# import pandas as pd

# # Carregar modelos
# detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
# labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
# labels = labels['OBJECT (2017 REL.)']

# cap = cv2.VideoCapture(0)

# width = 512
# height = 512

# while(True):
#     #Capture frame-by-frame
#     ret, frame = cap.read()
    
#     #Resize to respect the input_shape
#     inp = cv2.resize(frame, (width , height ))

#     #Convert img to RGB
#     rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

#     #Is optional but i recommend (float convertion and convert img to tensor image)
#     rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

#     #Add dims to rgb_tensor
#     rgb_tensor = tf.expand_dims(rgb_tensor , 0)
    
#     boxes, scores, classes, num_detections = detector(rgb_tensor)
    
#     pred_labels = classes.numpy().astype('int')[0]
    
#     pred_labels = [labels[i] for i in pred_labels]
#     pred_boxes = boxes.numpy()[0].astype('int')
#     pred_scores = scores.numpy()[0]
#     #loop throughout the faces detected and place a box around it
    
#     for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
#         if score < 0.5:
#             continue
            
#         score_txt = f'{100 * round(score,0)}'
#         img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),1)      
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         cv2.putText(img_boxes,label,(xmin, ymax-10), font, 0.5, (255,0,0), 1, cv2.LINE_AA)
#         cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 0.5, (255,0,0), 1, cv2.LINE_AA)



#     #Display the resulting frame
#     cv2.imshow('black and white',img_boxes)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()


# Static Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
path = "/content/drive/MyDrive/AdelKhoramruz"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)
np.asarray(dir_list).shape

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
MyHyper=0
for pic in dir_list:
  img = cv2.imread('/content/drive/MyDrive/AdelKhoramruz/{}'.format(pic))
  if img.shape[0] > MyHyper:
    MyHyper = img.shape[0]
    print(MyHyper)
  else:
    pass

In [ ]:
import tensorflow_hub as hub
import cv2
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# Apply image detector on a batch of image.
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")


width = 1028
height = 1028

labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']
z = 0
for pic in dir_list:
  #Load image by Opencv2
  img = cv2.imread('/content/drive/MyDrive/AdelKhoramruz/{}'.format(pic))
  #Resize to respect the input_shape
  inp = cv2.resize(img, (width , height ))

  #Convert img to RGB
  rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

  #Is optional but i recommend (float convertion and convert img to tensor image)
  rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

  #Add dims to rgb_tensor
  rgb_tensor = tf.expand_dims(rgb_tensor , 0)

  #Now you can use rgb_tensor to predict label for exemple :

  boxes, scores, classes, num_detections = detector(rgb_tensor)

  pred_labels = classes.numpy().astype('int')[0] 
  pred_labels = [labels[i] for i in pred_labels]
  pred_boxes = boxes.numpy()[0].astype('int')
  pred_scores = scores.numpy()[0]

  for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
          if score < 0.4:
            continue
          z+=1
          score_txt = f'{100 * round(score)}%'
          img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),2)      
          font = cv2.FONT_HERSHEY_SIMPLEX
          cv2.putText(img_boxes, label,(xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
          crop_img = img_boxes[ymin:ymax, xmin:xmax]
          crop_img = image_resize(crop_img, height = MyHyper)
          crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
          crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
          (thresh, crop_img) = cv2.threshold(crop_img, 150, 255, cv2.THRESH_BINARY)
          plt.imshow(crop_img)
          plt.savefig('/content/drive/MyDrive/adel_final/{}.jpg'.format(z))
          cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
